Import streamlit,pyngrok, and ngrok modules

In [1]:
!pip install streamlit
!pip install pyngrok===4.1.1
!pip install plotly
from pyngrok import ngrok

     |████████████████████████████████| 8.0 MB 5.2 MB/s 
     |████████████████████████████████| 4.2 MB 50.6 MB/s 
     |████████████████████████████████| 170 kB 57.5 MB/s 
     |████████████████████████████████| 75 kB 4.3 MB/s 
     |████████████████████████████████| 111 kB 47.4 MB/s 
     |████████████████████████████████| 63 kB 1.9 MB/s 
     |████████████████████████████████| 122 kB 53.0 MB/s 
     |████████████████████████████████| 786 kB 45.3 MB/s 
     |████████████████████████████████| 370 kB 56.3 MB/s 
  Created wheel for blinker: filename=blinker-1.4-py3-none-any.whl size=13478 sha256=bbffcec5e647ac6aa8349bd3ed48d0e35c097881874903965eb6dbe91b1e037d
  Stored in directory: /root/.cache/pip/wheels/22/f5/18/df711b66eb25b21325c132757d4314db9ac5e8dabeaf196eab
Successfully built blinker
  Attempting uninstall: prompt-toolkit
    Found existing installation: prompt-toolkit 1.0.18
    Uninstalling prompt-toolkit-1.0.18:
      Successfully uninstalled prompt-toolkit-1.0.18
  Attempting

  Created wheel for pyngrok: filename=pyngrok-4.1.1-py3-none-any.whl size=15984 sha256=b8daf0d87bed67e7cda0229545ea70f079b9b9ace90d559036bcc3995b76ef62
  Stored in directory: /root/.cache/pip/wheels/b1/d9/12/045a042fee3127dc40ba6f5df2798aa2df38c414bf533ca765
Successfully built pyngrok


In [3]:
!ngrok authtoken 1wrCPm9O476DJaOBDl7bBXqwGIf_5VAxUTeShyY8fV1NkFjCG

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns

df=pd.read_csv("/content/Final Data.csv")
df.head()

,Unnamed: 0,page,psex,ss_fev,ss_cou,ss_dys,ss_fa,co_dime,co_bp,co_kidis,co_lidis,co_cadis,co_obs,co_ild,co_imm,co_alc,co_smk,ss_oxsa,img_xray,tr_chis,po_po
0,0,27,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,98,0,0,0
1,1,55,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,98,0,1,0
2,3,27,2,1,1,0,0,0,0,0,0,0,0,0,0,0,0,98,1,1,0
3,4,33,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,98,1,0,0
4,5,58,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,100,0,1,0


Features and Outcome

In [ ]:
# define dataset
X = df[['page','psex','ss_fev','ss_cou','ss_dys','ss_fa','co_dime','co_bp','co_kidis','co_lidis','co_cadis','co_obs','co_ild','co_imm','co_alc','co_smk','ss_oxsa','img_xray','tr_chis']]
Y = df[['po_po']]

Splitting the data into train and test sets

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(X,Y)

Create a function for evaluating metrics

In [ ]:
from sklearn.metrics import accuracy_score,recall_score,f1_score,precision_score,roc_auc_score,confusion_matrix

def evaluation(Y_test,Y_pred):
  acc=accuracy_score(Y_test,Y_pred)
  rcl=recall_score(Y_test,Y_pred)
  f1=f1_score(Y_test,Y_pred)
 

  metric_dict={'accuracy': round(acc,3),
               'recall': round(rcl,3),
               'F1 score': round(f1,3),
               
              }

  return print(metric_dict)

Fitting XGB Model

In [ ]:
from xgboost import XGBClassifier
XGB_clf=XGBClassifier()
XGB_clf.fit(X_train.values,Y_train.values)
XGB_score=XGB_clf.score(X_test.values,Y_test.values)
XGB_Y_pred=XGB_clf.predict(X_test.values)
XGB_Y_prob=XGB_clf.predict_proba(X_test.values)
#print(SVC_score)
evaluation(Y_test.values,XGB_Y_pred)

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


{'accuracy': 0.982, 'recall': 0.211, 'F1 score': 0.308}


Tuning XGBoost manually

In [ ]:
xgb = XGBClassifier(learning_rate=0.01, n_estimators=25, max_depth=15,gamma=0.6, subsample=0.52,colsample_bytree=0.6,seed=27, 
                    reg_lambda=2, booster='dart', colsample_bylevel=0.6, colsample_bynode=0.5)

xgb.fit(X_train.values,Y_train.values)
xgb_score=XGB_clf.score(X_test.values,Y_test.values)
xgb_Y_pred=XGB_clf.predict(X_test.values)
xgb_Y_prob=XGB_clf.predict_proba(X_test.values)
#print(SVC_score)
evaluation(Y_test.values,xgb_Y_pred)

{'accuracy': 0.982, 'recall': 0.211, 'F1 score': 0.308}


/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Let's save our model using pickle

In [ ]:
import pickle as pkl
pkl.dump(XGB_clf,open("final_model.p","wb"))

In [ ]:
import sklearn
sklearn_version = sklearn.__version__
print(sklearn_version)

0.22.2.post1


Write a file for creating web app

In [106]:
%%writefile covitality-app.py
import streamlit as st
import base64
import sklearn
import pandas as pd
import numpy as np
import pickle as pkl
import plotly.graph_objects as go
from csv import writer
from tabulate import tabulate
from sklearn.preprocessing import MinMaxScaler
scal=MinMaxScaler()
#Load the saved model
#model=pkl.load(open("final_model.p","rb"))



st.set_page_config(page_title="COVITALITY APP",page_icon="⚕️",layout="centered",initial_sidebar_state="expanded")

st.sidebar.header('User Input Parameters')

# load spreadsheet with data to be annotated
def get_data():
  return []

def score(risk):
    
    if risk<15:
      report = '<p style="font-family:sans-serif; color:Green; font-size: 20px;">Low Mortality Risk</p>'
      st.markdown(report, unsafe_allow_html=True)
      st.write("Mortality Risk Score is ",risk)
      #st.write("Low Mortality Risk")
    elif risk >= 15 and risk <=25:
      report = '<p style="font-family:sans-serif; color:Yello; font-size: 20px;">Moderate Mortality Risk</p>'
      st.markdown(report, unsafe_allow_html=True)
      st.write("Mortality Risk Score is ",risk)
      #st.write("Moderate Mortality Risk")
    else :
      report = '<p style="font-family:sans-serif; color:Red; font-size: 20px;">High Mortality Risk</p>'
      st.markdown(report, unsafe_allow_html=True)
      st.write("Mortality Risk Score is ",risk)
      #st.write("High Mortality Risk")


def preprocess(page,psex,ss_fev,ss_cou,ss_dys,ss_fa,co_dime,co_bp,co_kidis,co_lidis,co_cadis,co_obs,co_ild,co_imm,co_alc,co_smk,ss_oxsa,img_xray,tr_chis):   
 
    risk = 0
    # Pre-processing user input   
    if page<50:
          risk = risk+1
    elif page >= 50 and page <=59:
          risk = risk+2
    else :
          risk = risk+3

    if psex=="male":
          risk = risk+2
    else: risk = risk+1

    if ss_fev=="Present":
          risk = risk+2
    else: risk = risk+0

    if ss_cou=="Present":
          risk = risk+2
    else: risk = risk+0

    if ss_dys=="Present":
          risk = risk+2
    else: risk = risk+0

    if ss_fa=="Present":
          risk = risk+2
    else:  risk = risk+0

    if co_dime=="Present":
          risk = risk+2
    else:  risk = risk+0

    if co_bp=="Present":
          risk = risk+3
    else: risk = risk+1

    if co_kidis=="Present":
          risk = risk+3
    else:  risk = risk+1

    if co_lidis=="Present":
          risk = risk+3
    else:  risk = risk+1

    if co_cadis=="Present":
          risk = risk+3
    else:  risk = risk+1

    if co_obs=="Present":
          risk = risk+3
    else:  risk = risk+0

    if co_ild=="Present":
          risk = risk+3
    else:  risk = risk+0

    if co_imm=="Yes":
          risk = risk+3
    else:  risk = risk+0

    if co_alc=="Yes":
          risk = risk+3
    else:  risk = risk+0

    if co_smk=="Yes":
          risk = risk+3
    else:  risk = risk+1

    if ss_oxsa<90:
          risk = risk+4
    elif ss_oxsa >= 90 and ss_oxsa <=95:
          risk = risk+2
    else :
          risk = risk+0

    if img_xray=="Abnormal":
          risk = risk+3
    else: risk = risk+2

    if tr_chis=="Yes":
          risk = risk+1
    else: risk = risk+0


    
    #user_input=np.array(user_input)
    #user_input=user_input.reshape(1,-1)
    #user_input=scal.fit_transform(user_input)
    return risk

    

       
    # front end elements of the web page 
html_temp = """ 
    <div style ="background-color:#7D0552;padding:13px"> 
    <h1 style ="color:white;text-align:center;">COVITALITY APP</h1> 
    </div> 
    """
      
# display the front end aspect
st.markdown(html_temp, unsafe_allow_html = True) 
st.subheader('by MANUSREE BANERJEE ')
      
# following lines create boxes in which user can enter data required to make prediction
pname = st.sidebar.text_input('Patient Name')
pid = st.sidebar.text_input('Patient ID')
page=st.sidebar.slider("Age",1,100)
psex = st.sidebar.radio("Select Gender: ", ('male', 'female'))
ss_fev = st.sidebar.selectbox('Fever',("Present","Absent")) 
ss_cou = st.sidebar.selectbox('Cough', ("Present","Absent"))
ss_dys = st.sidebar.selectbox('Dyspnea', ("Present","Absent"))
ss_fa = st.sidebar.selectbox('Fatigue', ("Present","Absent"))
co_dime = st.sidebar.selectbox('Diabetes Mellitus', ("Present","Absent"))
co_bp = st.sidebar.selectbox('Hypertension', ("Present","Absent"))
co_kidis = st.sidebar.selectbox('Chronic kidney disease', ("Present","Absent"))
co_lidis = st.sidebar.selectbox('Chronic liver disease', ("Present","Absent"))
co_cadis = st.sidebar.selectbox('Chronic cardiac disease', ("Present","Absent"))
co_obs = st.sidebar.selectbox('Obesity', ("Present","Absent"))
co_ild = st.sidebar.selectbox('Chronic pulmonary disease ', ("Present","Absent"))
co_imm = st.sidebar.selectbox('Is patient on Immunosuppressant / Steroids ?', ("Yes","No"))
co_alc = st.sidebar.selectbox('Did patient drink alcohol within past 5 years ?', ("Yes","No"))
co_smk = st.sidebar.selectbox('Did patient smoke within past 5 years ?', ("Yes","No"))
ss_oxsa = st.sidebar.slider('Oxygen Saturation',0,100)
img_xray = st.sidebar.selectbox('X-ray report ', ("Normal","Abnormal"))
tr_chis = st.sidebar.selectbox('Contact/Travel history', ("Yes","No"))



#user_input=preprocess(psex,ss_fev,ss_cou,ss_dys,ss_fa,co_dime,co_bp,co_kidis,co_lidis,co_cadis,co_obs,co_ild,co_imm,co_alc,co_smk,img_xray,tr_chis)
#pred=preprocess(page,psex,ss_fev,ss_cou,ss_dys,ss_fa,co_dime,co_bp,co_kidis,co_lidis,co_cadis,co_obs,co_ild,co_imm,co_alc,co_smk,ss_oxsa,img_xray,tr_chis)
#probability_class_1 = pred.item(1)
#prob = round(probability_class_1*100,1)

risk = preprocess(page,psex,ss_fev,ss_cou,ss_dys,ss_fa,co_dime,co_bp,co_kidis,co_lidis,co_cadis,co_obs,co_ild,co_imm,co_alc,co_smk,ss_oxsa,img_xray,tr_chis)

if st.button("Risk Strata Key"):
  # display table
  fig=go.Figure(data=go.Table(header=dict(values=['Risk Category', 'Risk Score'],fill_color='#FD8E72',font_size=18,align='center'),
                            cells=dict(values=[['Low','Moderate','High'],['10-14','15-25','25-50']],fill_color='#F9B7FF',font_color=['black'],font_size=14,align='center')))

  #fig.update_layout(autosize=True,margin=dict(l=0,r=0,b=0,t=0))
  fig.update_layout(autosize=False,
                  width=800,
                  height=350
                 )
  st.write(fig)

if st.button("Predict"):    
  mortality = score(risk)
  st.info("Don't forget to rate this app")
  
feedback = st.slider('How much would you rate this app?',min_value=0,max_value=5,step=1)
  
if feedback:
    st.header("Thank you for rating the app!")

#if st.button("Predict"):    
 # if pred[0] == 0:
  #  st.error('You have higher chances of surviving COVID-19.')
    
  #else:
   # st.success('Warning! You have lower chances of surviving COVID-19!')
    
   

st.subheader("About App")

st.info("This web app is helps you to find out whether you are at a risk of mortality from COVID-19.")
st.info("Enter the required fields and click on the 'Predict' button to check your survival chances from COVID-19.")
st.info("Build using Extreme Gradient Boosting Model with AUC score of 94%.")


st.caption("Caution: This is just a prediction and not doctoral advice. Kindly see a doctor if you feel the symptoms persist.") 


Overwriting covitality-app.py


In [ ]:
!ls

covitality-app.py  sample_data


In [108]:
!streamlit run covitality-app.py &>/dev/null&

In [109]:
from pyngrok import ngrok
# Setup a tunnel to the streamlit port 8501
public_url = ngrok.connect(port='8501')
public_url

'http://ee3e-146-148-44-202.ngrok.io'

In [107]:
ngrok.kill()